In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('database.csv', index_col = 0)
df

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,fee_cleaned,league_name,year,...,Goals-PKs/90,Goals+Assists-PKs/90,xG,npxG,xA,xG/90,xA/90,xG+xA/90,npxG/90,npxG+xA/90
0,Feyenoord Rotterdam,Steven Berghuis,25,Right Winger,Watford,£5.85m,in,5.85,Eredivisie,2017,...,0.30,0.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feyenoord Rotterdam,Ridgeciano Haps,24,Left-Back,AZ Alkmaar,£5.40m,in,5.40,Eredivisie,2017,...,0.00,0.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Feyenoord Rotterdam,Sofyan Amrabat,20,Central Midfield,FC Utrecht,£3.60m,in,3.60,Eredivisie,2017,...,0.00,0.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACF Fiorentina,Sofyan Amrabat,23,Central Midfield,Hellas Verona,£9.49m,in,9.49,Serie A,2019,...,0.08,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hellas Verona,Sofyan Amrabat,23,Central Midfield,Club Brugge,£3.15m,in,3.15,Serie A,2019,...,0.08,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,CA Osasuna,Marc Cardona,23,Centre-Forward,FC Barcelona,£2.25m,in,2.25,Primera Division,2019,...,0.45,0.60,2.7,2.7,0.6,0.41,0.10,0.50,0.41,0.50
1605,CA Osasuna,Robert Ibáñez,26,Right Winger,Getafe,£1.80m,in,1.80,Primera Division,2019,...,0.00,0.00,0.2,0.2,0.0,0.25,0.00,0.25,0.25,0.25
1606,Granada CF,Darwin Machís,26,Left Winger,Udinese Calcio,£2.70m,in,2.70,Primera Division,2019,...,0.00,0.19,1.3,1.3,0.6,0.26,0.12,0.38,0.26,0.38
1607,Granada CF,Domingos Duarte,24,Centre-Back,Sporting CP,£2.70m,in,2.70,Primera Division,2019,...,0.10,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609 entries, 0 to 1608
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   club_name             1609 non-null   object 
 1   player_name           1609 non-null   object 
 2   age                   1609 non-null   int64  
 3   position              1609 non-null   object 
 4   club_involved_name    1609 non-null   object 
 5   fee                   1609 non-null   object 
 6   transfer_movement     1609 non-null   object 
 7   fee_cleaned           1609 non-null   float64
 8   league_name           1609 non-null   object 
 9   year                  1609 non-null   int64  
 10  season                1609 non-null   object 
 11  links                 1609 non-null   object 
 12  Season                1533 non-null   object 
 13  Matches               1533 non-null   float64
 14  Starts                1533 non-null   float64
 15  Minutes              

In [4]:
df.xG.fillna(df.Goals, inplace = True)
df.npxG.fillna(df.Goals - df.Penalty_Goals, inplace = True)
df.xA.fillna(df.Assists, inplace = True)
df['xG/90'].fillna(df['Goals/90'], inplace = True)
df['xA/90'].fillna(df['Assists/90'], inplace = True)
df['xG+xA/90'].fillna(df['Goals+Assists/90'], inplace = True)
df['npxG/90'].fillna(df['Goals-PKs/90'], inplace = True)
df['npxG+xA/90'].fillna(df['Goals+Assists-PKs/90'], inplace = True)

In [5]:
df.dropna(subset = ['Season'], inplace = True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1533 entries, 0 to 1608
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   club_name             1533 non-null   object 
 1   player_name           1533 non-null   object 
 2   age                   1533 non-null   int64  
 3   position              1533 non-null   object 
 4   club_involved_name    1533 non-null   object 
 5   fee                   1533 non-null   object 
 6   transfer_movement     1533 non-null   object 
 7   fee_cleaned           1533 non-null   float64
 8   league_name           1533 non-null   object 
 9   year                  1533 non-null   int64  
 10  season                1533 non-null   object 
 11  links                 1533 non-null   object 
 12  Season                1533 non-null   object 
 13  Matches               1533 non-null   float64
 14  Starts                1533 non-null   float64
 15  Minutes              

In [7]:
df.fillna(0, inplace = True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1533 entries, 0 to 1608
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   club_name             1533 non-null   object 
 1   player_name           1533 non-null   object 
 2   age                   1533 non-null   int64  
 3   position              1533 non-null   object 
 4   club_involved_name    1533 non-null   object 
 5   fee                   1533 non-null   object 
 6   transfer_movement     1533 non-null   object 
 7   fee_cleaned           1533 non-null   float64
 8   league_name           1533 non-null   object 
 9   year                  1533 non-null   int64  
 10  season                1533 non-null   object 
 11  links                 1533 non-null   object 
 12  Season                1533 non-null   object 
 13  Matches               1533 non-null   float64
 14  Starts                1533 non-null   float64
 15  Minutes              

In [9]:
df.Minutes = df.Minutes.apply(lambda x: str(x).replace(',', ''))

In [10]:
df.Minutes = df.Minutes.astype('int64')

In [11]:
df_features = df[['age','position','Starts','Minutes','Goals','Assists','Penalty_Goals','Penalty_Attempts',
                  'Yellow_Cards','Red_Cards','Goals/90','Assists/90','Goals+Assists/90','Goals-PKs/90',
                  'Goals+Assists-PKs/90','xG','npxG','xA','xG/90','xA/90','xG+xA/90','npxG/90','npxG+xA/90']]

In [27]:
df_dummies = pd.get_dummies(df_features.position)
df_features = pd.concat([df_features, df_dummies], axis = 1)
df_features.drop(['position'], axis = 1, inplace = True)

In [28]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1533 entries, 0 to 1608
Data columns (total 61 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   age                   1533 non-null   int64  
 1   Starts                1533 non-null   float64
 2   Minutes               1533 non-null   int64  
 3   Goals                 1533 non-null   float64
 4   Assists               1533 non-null   float64
 5   Penalty_Goals         1533 non-null   float64
 6   Penalty_Attempts      1533 non-null   float64
 7   Yellow_Cards          1533 non-null   float64
 8   Red_Cards             1533 non-null   float64
 9   Goals/90              1533 non-null   float64
 10  Assists/90            1533 non-null   float64
 11  Goals+Assists/90      1533 non-null   float64
 12  Goals-PKs/90          1533 non-null   float64
 13  Goals+Assists-PKs/90  1533 non-null   float64
 14  xG                    1533 non-null   float64
 15  npxG                 

In [29]:
df_target = df.fee_cleaned

In [30]:
X_train, X_test, y_train, y_test = train_test_split(df_features, df_target)

In [33]:
line = LinearRegression(n_jobs = -1)
line.fit(X_train, y_train)
y_train_preds = line.predict(X_train)
mean_squared_error(y_train, y_train_preds)

135.62527001820558

In [34]:
y_preds = line.predict(X_test)
mean_squared_error(y_test, y_preds)

144.79501625962192

In [36]:
ridge = Ridge()
ridge.fit(X_train, y_train)
y_train_preds = ridge.predict(X_train)
mean_squared_error(y_train, y_train_preds)

137.8885699918143

In [37]:
y_preds = ridge.predict(X_test)
mean_squared_error(y_test, y_preds)

136.7887089755791